In [ ]:
# Set Jupyter to render directly to the screen
%matplotlib inline

# Import pandas and numpy for analysis
import pandas as pd
import numpy as np

# Reading in the CSV file

In [ ]:
def read_data(filename):
    '''
    This reads the .csv stored at the 'filename' location and returns a DataFrame
    with two-level columns. The first level column contains the Exchange and the 
    second contains the type of market data, e.g. bid/ask, price/volume.
    '''
    df = pd.read_csv(filename, index_col=0)
    df.columns = [df.columns.str[-7:], df.columns.str[:-8]]

    return df

In [ ]:
# Read the data for one of the stocks (HWG)

filename = 'HWG.csv'
market_data = read_data(filename)

# Some Examples

In [ ]:
# What is in the dataframe? (Display top 5 rows with the .head() DataFrame method.)

market_data.head()

In [ ]:
# Extracting the BidVolumes on Z-XHCHNG. Note how the printed value looks different 
# from the values printed for the DataFrame, as the single column is a Series.

bid_volumes_Z = market_data['Z-XCHNG', 'BidVolume']
bid_volumes_Z.head()

In [ ]:
# Extracting the BidVolume at a specific time on Z-XHCHNG. This is just a number (float).

time = '2018-01-05 10:20:00'

bid_volume_Z_at_time = market_data.loc[time, ('Z-XCHNG', 'BidVolume')]
bid_volume_Z_at_time

In [ ]:
# Extracting a subset of observations, here the 1220th to the 1230th.

market_data.iloc[1220:1230]

In [ ]:
# Adding a new column based on a calculation of old columns

bid_ask_spread_I = market_data['I-XCHNG', 'AskPrice'] - market_data['I-XCHNG', 'BidPrice']
bid_ask_spread_Z = market_data['Z-XCHNG', 'AskPrice'] - market_data['Z-XCHNG', 'BidPrice']

market_data['I-XCHNG', 'BidAskSpread'] = bid_ask_spread_I
market_data['Z-XCHNG', 'BidAskSpread'] = bid_ask_spread_Z

market_data.head()

In [ ]:
# The resulting DataFrame still looks a bit disordered, it added the new columns to the end.
# We can alphabetically sort the columns to clean things up a bit.

market_data = market_data.sort_index(axis=1)
market_data.head()

In [ ]:
# If we want to iterate over each timestamp, we can easily do so.
# Let's calculate the maximum BidAskSpread on I-XCHNG seen in the whole dataset.

max_spread_seen = 0
max_spread_seen_time = None

for time, mkt_data_at_time in market_data.iterrows():
    spread = mkt_data_at_time['I-XCHNG', 'BidAskSpread'] 
    
    if spread > max_spread_seen:
        max_spread_seen = spread
        max_spread_seen_time = time

print(max_spread_seen_time, max_spread_seen)

In [ ]:
# Now let's do a comparison between different timestamps.
# We will calculate the maximum price-increase of the I-XCHNG BidPrice for the whole dataset.

max_price_increase = -999999
max_price_increase_seen_time = None

prev_time = None

for time, mkt_data_at_time in market_data.iterrows():
    if prev_time == None:
        # Skip the first observation, there is no previous bid price yet.
        prev_time = time
        continue
    
    previous_bid_price = market_data.loc[prev_time, ('I-XCHNG', 'BidPrice')]  
    current_bid_price = mkt_data_at_time['I-XCHNG', 'BidPrice']
       
    bid_price_increase = current_bid_price - previous_bid_price
    
    if bid_price_increase > max_price_increase:
        max_price_increase = bid_price_increase
        max_price_increase_seen_time = time
        
    # Update the previous time for the next iteration of the loop.
    prev_time = time

print(max_price_increase_seen_time, max_price_increase)

# Exercise

In [ ]:
# Now it's your turn. Start with plotting some of the values in the market_data DataFrame.
# (Hint: It's a DataFrame method!)